In [1]:
import pandas as pd
import numpy as np

In [2]:
import dask
from dask.distributed import Client
from dask import dataframe as dd
import sys

print("Python Version : ", sys.version)
print("Dask Version : ", dask.__version__)

client = Client(n_workers=2, threads_per_worker=4)
client

Python Version :  3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Dask Version :  2021.12.0


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 35.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45349,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 35.19 GiB
Comm: tcp://127.0.0.1:40617,Total threads: 4
Dashboard: http://127.0.0.1:43321/status,Memory: 17.59 GiB
Nanny: tcp://127.0.0.1:39679,


### Data

In [3]:
data_len = 30000000
id_list = [i for i in range(data_len)]
df1 = pd.DataFrame({'id': id_list,
                    'value': np.random.randint(0,100,data_len)})
df2 = pd.DataFrame({'id': id_list,
                    'value': np.random.randint(0,100,data_len)})

### Pandas

In [4]:
%time df1.merge(df2, on ='id', how='left')

CPU times: user 8.63 s, sys: 1.77 s, total: 10.4 s
Wall time: 10 s


,id,value_x,value_y
0,0,76,90
1,1,42,97
2,2,30,74
3,3,69,1
4,4,45,75
...,...,...,...
29999995,29999995,50,98
29999996,29999996,58,47
29999997,29999997,33,83
29999998,29999998,6,18


### Dask

In [5]:
dd1 = dask.dataframe.from_pandas(df1, npartitions=2)
dd2 = dask.dataframe.from_pandas(df2, npartitions=2)

In [6]:
%time dd1.merge(dd2, on ='id', how='left').compute()

CPU times: user 2.63 s, sys: 2.88 s, total: 5.51 s
Wall time: 18.3 s


,id,value_x,value_y
0,1,42,97
1,9,99,75
2,10,77,56
3,11,46,36
4,13,42,75
...,...,...,...
15004201,29999980,12,58
15004202,29999986,6,38
15004203,29999987,11,91
15004204,29999988,13,98
